Load Environment variables
--------------------------

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "multi-agent-search-bot"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


#Import Library for tools:

from typing import Annotated
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL


#Import Library for Agent Supervisor:

from typing import Literal
from typing_extensions import TypedDict

from langchain_groq import ChatGroq
from langgraph.graph import MessagesState
from langgraph.types import Command
from langchain_core.messages import  trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langgraph.graph.message import add_messages


Create tools:
-------------
We will create tool to do web research with a search engine using Tavily Search, and another tool to do math operations and create plots using Python Repl.

In [10]:

tavily_tool = TavilySearchResults(max_results=5)

repl = PythonREPL()

@tool
def python_rep_tool(code: Annotated[str, "Python code to execute to generate your chart."]) :
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n '''python\n{code}\n'''\nStdout: {result}"
    return result_str

Create Agent Supervisor:
------------------------
We will use LLM to decide whether to choose the next worker node OR Finish processing.

In [11]:
members = ["researcher", "coder"]


system_prompt =(
    f"You are a supervisor tasked with managing a conversation between the following workers: {members}. Your role is to route the user's request to the appropriate workers based on the task at hand."
    "Here are the rules you must follow:"
    f"1. **Routing to Workers**: When a user request requires a specific worker, respond with the name of the worker {members} to route the request to that worker."
    "2. **Handling Results**: After receiving results from a worker, evaluate whether the user's request has been fully addressed. If the request is complete, respond with the keyword FINISH to indicate the end of the workflow. If further actions are needed, route the request to another worker or back to the same worker as necessary."
    "3. **Final Response**: When you decide to end the workflow, ensure your response is with their results ."
    " Ensure all tool calls are formatted correctly."
    )
        

class Router(TypedDict):
    f"""Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal["researcher", "coder","FINISH"]


llm=ChatGroq(groq_api_key=os.environ["GROQ_API_KEY"],model="mixtral-8x7b-32768",temperature=0, max_tokens=200)


class State(MessagesState):
    next: str


Supervisor node (LLM) that decides which agent worker nodes should be called next, based on the user input query.


In [12]:

def supervisor_node(state: State) -> Command[Literal["researcher", "coder", "__end__"]]:
        messages = [
            {"role": "system", "content": system_prompt},
            ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        print('llm-response:',response)

        goto = response["next"]
        print("goto:",goto)

        if goto == "FINISH":
            goto = END
        return Command(goto=goto,update={"next": goto})

Construct Graph
----------------
We will start building the graph. First, we will define the state and worker nodes function that we defined above.

In [13]:
 #Search_Tool
research_agent = create_react_agent(
    llm , tools=[tavily_tool], state_modifier="You are a researcher. DO NOT do any math operations.")


def research_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
        ]
        },
        goto="supervisor",
    )


# Math_operator_Tool
code_agent = create_react_agent(llm, tools=[python_rep_tool], state_modifier="You are a Math Operator Tool.do the math operations only.")


def code_node(state: MessagesState) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )

we will define the Graph workflow:

In [14]:
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
graph = builder.compile()


Invoke the Graph
-----------------
With the graph created, we can now invoke it and see how it performs!

In [15]:
def call_agent(user_input):
    output = []
    for s in graph.stream({"messages": [("user", user_input)]}, subgraphs=True):
        print(s)
        print("----")
        output.append(s)
    
    a = output [-2][1]
    #print(a)

    if 'researcher' in a:
        #print("The key 'researcher' is present.")
        output1 = a['researcher']['messages'][-1].content

    elif 'coder'in a:
        #print("The key 'coder' is present.")
        output1 = a['coder']['messages'][-1].content

    return output1
    

In [16]:
b = call_agent("what is 48946 * 54198") 

llm-response: {'next': 'coder'}
goto: coder
((), {'supervisor': {'next': 'coder'}})
----


Python REPL can execute arbitrary code. Use with caution.


(('coder:dea511d3-2c4c-2411-eece-430c6f339d7c',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_97tm', 'function': {'arguments': '{"code":"result = 48946 * 54198\\nprint(result)"}', 'name': 'python_rep_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 1244, 'total_tokens': 1354, 'completion_time': 0.168634911, 'prompt_time': 0.081620041, 'queue_time': 0.021440709000000002, 'total_time': 0.250254952}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bac77f35-de83-4409-bdb0-7eb731a27339-0', tool_calls=[{'name': 'python_rep_tool', 'args': {'code': 'result = 48946 * 54198\nprint(result)'}, 'id': 'call_97tm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1244, 'output_tokens': 110, 'total_tokens': 1354})]}})
----
(('coder:dea511d3-2c4c-2411-eece-430c6f339d7c',), {'tools': {'messages': [ToolMessa

In [20]:
b

'The product of 48946 and 54198 is 2652775308.'

In [17]:
c = call_agent("what is the square of 457587")

llm-response: {'next': 'coder'}
goto: coder
((), {'supervisor': {'next': 'coder'}})
----
(('coder:54c111a8-778e-df4b-6af7-9da276e0bb30',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pr2h', 'function': {'arguments': '{"code":"result = 457587 ** 2; print(result)"}', 'name': 'python_rep_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 1241, 'total_tokens': 1347, 'completion_time': 0.162051032, 'prompt_time': 0.067271472, 'queue_time': 0.020980227000000004, 'total_time': 0.229322504}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6489b3b5-4c78-4ad9-b7fd-436b52225245-0', tool_calls=[{'name': 'python_rep_tool', 'args': {'code': 'result = 457587 ** 2; print(result)'}, 'id': 'call_pr2h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1241, 'output_tokens': 106, 'total_tokens': 1347})]}})
----

In [21]:
c

'The square of 457587 is 209385862569.'

In [26]:
d = call_agent("what is the weather in chennai?") 

llm-response: {'next': 'researcher'}
goto: researcher
((), {'supervisor': {'next': 'researcher'}})
----
(('researcher:d3787902-ccfa-d24e-26e3-e88addb5d157',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_en3q', 'function': {'arguments': '{"query":"weather in chennai"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 1232, 'total_tokens': 1333, 'completion_time': 0.15464946, 'prompt_time': 0.081221259, 'queue_time': 0.020916042999999995, 'total_time': 0.235870719}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2f3be568-f47f-4688-8f53-766ae96c9f12-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in chennai'}, 'id': 'call_en3q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1232, 'output_tokens': 101, 'total_tokens': 1333})

In [27]:
d

'The weather in Chennai, India in February is partly cloudy with an average temperature of around 28°C. The temperature can range from a minimum of 26°C to a maximum of 31°C. There is typically no rainfall during this month.'

In [19]:
e = call_agent("what is the gold price in chennai?") 

llm-response: {'next': 'researcher'}
goto: researcher
((), {'supervisor': {'next': 'researcher'}})
----
(('researcher:6c99d265-69a3-84ac-5bb1-38553db82f9c',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0627', 'function': {'arguments': '{"query":"gold price in chennai"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 1233, 'total_tokens': 1352, 'completion_time': 0.183243277, 'prompt_time': 0.074245302, 'queue_time': 0.02091208, 'total_time': 0.257488579}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db4c98d4-6637-440c-a351-fd79c276f2ab-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'gold price in chennai'}, 'id': 'call_0627', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1233, 'output_tokens': 119, 'total_tokens': 1352})]}}

In [22]:
e

'The gold price in Chennai varies depending on the karat of the gold. For 24 karat gold, the price is around ₹ 8,449 per gram, while for 22 karat gold, the price is around ₹ 7,745 per gram. These prices may vary slightly depending on the source.'

In [23]:
f = call_agent("Who is the ICC men's cricketer of the year 2024?")

llm-response: {'next': 'researcher'}
goto: researcher
((), {'supervisor': {'next': 'researcher'}})
----
(('researcher:e6164dcd-0d93-8c48-7cee-139ff4882507',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_myhg', 'function': {'arguments': '{"query":"ICC men\'s cricketer of the year 2024"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 1242, 'total_tokens': 1352, 'completion_time': 0.168900081, 'prompt_time': 0.072791041, 'queue_time': 0.018969181, 'total_time': 0.241691122}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f0c0cb94-39ea-4477-956b-b544158e91c8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': "ICC men's cricketer of the year 2024"}, 'id': 'call_myhg', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1242, 'output_tokens'

In [24]:
f

"Jasprit Bumrah is the ICC Men's Cricketer of the Year 2024."